# Systems Programming

### Lecture 8: Dynamic Memory Management

### Anne Reinarz

anne.k.reinarz@durham.ac.uk


# Recap

Last lecture, we learned about **Pointers** and **Pointer Arithmetic**

```
int a[10];
int *pa;
```
These pairs of statements are equivalent using array or pointer notation:
```
pa = &a[0];
pa = a;
```
<center><img src="images/point2first.png" alt="point to array" width="600"/></center>

# Recap

Last lecture, we learned about **Pointer Arithmetic**

```
int a[10];
int *pa;
```
These pairs of statements are equivalent (+1 translates to +4 bytes (1 int)):
```
pa = &a[1];
pa = (a+1);
```
<center><img src="images/point2second.png" alt="point to array" width="600"/></center>

# Recap

We also learned about **Pointers to Pointers**.

```
int i, *p, **q;
p = &i;
q = &p;
```

<center><img src="images/pointer2pointer.png" alt="pointer to pointer" width="900"/></center>


- Now `i`, `*p` and `**q` all have value 10.

### and now...
# Pointer Safety

- Pointers can cause hard-to-diagnose errors in programs.
- e.g. a function returning pointers to local variables can cause errors when the memory is released back to the runtime system and reused at some time later in the program!

# Pointer Safety

- Always set pointers to `NULL` when they are no longer required
- Always use a simple guard before using pointers e.g.: `assert(ptr != NULL);` from `<assert.h>`
- There are various tools e.g. **Valgrind** (https://www.valgrind.org/) that can detect many such errors at runtime (at the cost of massive slowdown and greatly increased memory-usage)

# Dynamic Memory Allocation

- Variables and arrays provide fixed size allocation.
- What if the memory needed cannot be pre-determined?
- There is a need to be able to dynamically request variable sized blocks of memory from the runtime system.
- Close integration between C and Unix (and other Operating Systems).
- Requesting memory at run-time.

# Memory Layout 

<center><img src="images/memory-layout-ver.png" alt="memory layout" width="280"></center>

# Memory Layout 

### Stack:

- Stores “temporary” data
- Variables in a function
- Function header
- Small data

<center><img src="images/memory-layout-hor.png" alt="memory layout" width="700"></center>

# Memory Layout 

### Static Data:

- Data that stays in memory for the duration of the program.

<center><img src="images/memory-layout-hor.png" alt="memory layout" width="700"></center>

# Memory Layout 

### Heap:

- Used for storing more long-term data.
- YOU, the programmer, control what is in the Heap and when it is released.
- Much more space than the Stack.

<center><img src="images/memory-layout-hor.png" alt="memory layout" width="700"></center>

# Memory Allocation: `malloc()` <stdlib.h>

Function prototype for `malloc()`:

```c
void *malloc(size_t size);
```

- Allocates a contiguous block of memory `size` bytes long.

- The return type is `void*`, which is a generic pointer type that can be used with all types.

- `malloc()` returns a `NULL` pointer if it fails to allocate the requested memory.

# Memory Allocation: `malloc()` <stdlib.h>

```c
void *malloc(size_t size);
```

- Always test for `NULL` return in case `malloc()` has failed to allocate the requested memory!

- An Aside:

    - By default, Linux follows an optimistic memory allocation strategy.

    - Even if `malloc()` returns a non-`NULL` value, this is no guarantee that the memory is really available.

    - The operating system actually allocates the memory when you try to use it for the first time; if the system runs out of memory, a process will be killed by the Out Of Memory (OOM) killer.

# Example of `malloc()`

```c
#define SIZE 41 * sizeof(char)
char *line;
line = malloc(SIZE);
if ( line == NULL ) {
    printf( "Error in malloc() \n" );
    exit(1);}
```
N.B. return value of `malloc()` is automatically cast from `void*` to `char*`

- Pointers pointing to any object are automatically converted to `void*` pointers and vice versa as required.
- Conversions between other sorts of pointers with different types may cause problems due to alignment issues.

# Example of `malloc()`

- Creating an array using `malloc()` and printing values.
- The values are random.
- Freeing and allocating multiple times will produce different values.

In [1]:
#include <stdio.h>
#include <stdlib.h>

int main(){
    // allocate a 20 int array
    int *a =  malloc(20 * sizeof(int));
    if (a == NULL) exit(1);
    int i;
    for(i = 0; i < 20; i++){
        printf("%d\n",a[i]);
    }
    return 0;
}

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


# Memory De-allocation: `free()` <stdlib.h>

```
void free(void *ptr);
```

- Takes a generic pointer to a block of memory and returns the memory for reuse by `malloc()`.

- If you “forget” about memory you have `malloc()`ed and don’t `free()` it then you have a **“memory leak”**.

- “Memory leaks” can be very dangerous and difficult to trace [garbage collection in Java/Python].

    - Can eventually use up all memory

    - `free()` has no return value, so even if you pass it a pointer not allocated by `malloc()`, it will process it!

# Example of Memory Leak


In [2]:
#include<stdlib.h>
#include<stdio.h>

int main(){
    int i, *p;
    for(i=0; i<10000000; i++){
        p = malloc(10000000*sizeof(int));
        if (p == NULL){
            printf("Could not allocate more memory!");
            exit(1);
        }
        *p = i;
    }
    return 0;
}

[C kernel] Executable exited with code -9

In [3]:
#include <stdlib.h>
#include <stdio.h>

int main(void)
{ 
    char *line = NULL;
    size_t size = 0;
    for(;;) { 
        getline(&line, &size, stdin); // new memory implicitly allocated
        line = NULL;
    }
    return 0;
 }

/tmp/tmpy6df5dzv.c: In function ‘main’:
/tmp/tmpy6df5dzv.c:9:9: warning: implicit declaration of function ‘getline’ [-Wimplicit-function-declaration]
    9 |         getline(&line, &size, stdin); // new memory implicitly allocated
      |         ^~~~~~~
[C kernel] Executable exited with code -9

# Example of `free()`

```c
// allocate some memory
char *line = malloc(SIZE);

// use line in the program

free( line ); // return memory to the O/S
line = NULL;  // set pointer to NULL
```

- Errors from continuing to use a pointer after the memory has been released can be very hard to detect.
- N.B. line is implicitly cast to a `void*` pointer.

# Memory Allocation: `calloc()` <stdlib.h>

Function prototype for `calloc()`:

```c
void *calloc( size_t n, size_t size );
```

- Allocates a contiguous block of memory of `n` elements each of `size` bytes long, initialised to 0 in all bits.

- Useful to ensure old data is not reused inappropriately.

- The return type is `void*` - generic pointer type used for all types.

- `calloc()` returns a `NULL` pointer if it fails to allocate memory.

    - Always test for `NULL` return!

# `calloc()` Example

In [4]:
#include <stdio.h>
#include <stdlib.h>

int main(){
    //allocate an array of 20 ints
    //int *a = malloc(20 * sizeof(int));
    int *a = calloc(20, sizeof(int));
    if (a == NULL) exit(1);
    int i;
    for(i = 0; i < 20; i++){
        printf("%d\n",a[i]); // 0 because calloc zeros out allocated memory
    }
    return 0;
}

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


# Memory Allocation: `realloc()` <stdlib.h>

Function prototype for `realloc()`:

```c
void *realloc( void *ptr, size_t size );
```

- Allows a dynamic **change** in size of a previously allocated block of memory pointed to by `ptr`.

    - `ptr` must point to memory previously allocated by `malloc()`, `calloc()` or `realloc()`.

# Memory Allocation: `realloc()` <stdlib.h>

Function prototype for `realloc()`:

```c
void *realloc( void *ptr, size_t size );
```

- `realloc()` moves and copies contents if it needs to, freeing original block, which means `ptr` might change.

- `realloc()` returns a `NULL` pointer if it fails. Check for this!

    - Cf. `ArrayList` in **Java**.

# `realloc()` Example

Simple program that takes integers typed in by the user and stores them in an array.

- Each time the array becomes full, it is dynamically increased in size to hold more numbers

- Contains a key function `getline2()`, which reads the integers from the command line

```c
int getline2(char line[], int max) {
    int nch = 0;
    int c;
    max = max - 1;/* leave room for ’\0’ */
    while((c = getchar()) != ’q’) {
        if(c == ’\n’)
            break;
```
```c
        if(nch < max) {
            line[nch] = c;
            nch = nch + 1;
        }
    }
    if(c == ’q’ && nch == 0)
        return ’q’;

    line[nch] = ’\0’;
    return nch; 
}
```

# `realloc()` Example

### ```getline2()```
- Uses `getchar()` to read in characters as they are typed.
- Runs in a loop until a ’q’ or a newline is encountered.
- Reads in the characters typed by the user one by one and stores them in the array line.
- When the character ’\n’ is pressed, the function returns, via use of the break statement to exit a loop.
- No checking performed to see if the input is an integer.

```c
ip = malloc(array_size * sizeof(int));
while( getline2(line, MAXLINE) != ’q’ ) {
    if(nitems >= array_size ) {/* increase allocation */
        int *newp;
        array_size += INCREASE ;
        newp = realloc(ip, array_size * sizeof(int));
```
```c
        printf("<< Expanding by %d to size %d >>\n", INCREASE, array_size );
        if(newp == NULL) {
            printf("out of memory\n");
            exit(1);
        }
        ip = newp;
    }
    ip[nitems++] = atoi(line);
}
```

# `realloc()` Example

### ```main()```
- Uses `getline2()` to read in a line of text.
- Creates an array to store current line of text, `line`.

- Creates a second array to store the integers entered: `ip`.
- As soon as `ip` is full, `realloc()` is called to resize the array

# `atoi()` <stdlib.h>

```int atoi(const char *s);```

- Converts a string pointed to by `s` to an integer.

- Also see `atof()`, `atol()` and `atoll()` (since C99) equivalents

- To convert from an integer to a string use :

    `int sprintf( char *s, char *format, <value list> );`

    - Where the value list is the variables used in the format string.
    - `sprintf(str, "Sum of %d and %d is %d", a, b, a+b);`

# `->` Operator

The -> operator gives us a shorthand accessing members of structures using a pointer.

```c
struct point {
    int x;
    int y;
} pt, *ptr;
  
    ptr=&pt;
```

We can now modify `pt.x` in three ways:

```c
     pt.x=3; // Access directly
     (*ptr).x=3; // Access by dereferencing a pointer
     ptr->x=3;   // Access using the -> operator
````

# Summary

- Memory Allocation
- Memory Layout
- `malloc()`
- `free()`
- `calloc()`
- `realloc()`